# 4. How to use fixtures
> https://docs.pytest.org/en/8.0.x/how-to/fixtures.html

## 학습
- Fixtures란?
    - "provides a defined, reliable and consistent context for the tests"
- fixtures를 여러 개 넣어서 테스트 할 수도 있다.
- 하나의 테스트에서 여러 번 fixture를 호출할 경우, fixture는 재실행 되지 않고 기존 값이 사용된다.
- autouse fixtures
    - 직접 요청하지 않아도, 테스트에서 자동으로 요청되는 fixtures
- scope
    - 특정 범위 내에서 공용으로 사용할 fixture를 정의할 수 있다.
    - 유용성: 시간이 오래 소요되는 서버 연결 등을 최초 한 번만 수행하고 여러 테스트를 해당 연결을 사용해서 처리
- fixture finalization
    - pytest의 fixtures는 유용한 teardown 시스템을 가지고 있다. 즉 각 fixture가 사용된 후 clean up 하는 데 필요한 단계를 정의할 수 있다.
    - `yield` fixtures
        - return 대신 사용되며, yield 뒤에 `teardown code`가 작성된다.
        - 테스트가 끝나면 pytest는 fixture 리스트의 반대 순서로 순회하며 yielded된 fixture의 teardown code를 실행한다.
        - yield 전에 예외가 발생할 경우 teardown code는 실행되지 않는다.

## 테스트

### fixture 기본 활용
test_string()의 파라미터인 `order`와 같은 이름의 fixture를 찾아 test_string()에 넣어준다. 그리고 order의 경우에도 `first_entry`와 같은 이름의 fixture를 찾아 order()에 넣어준다.

In [8]:
# test_append
!pytest test_append.py

============================= test session starts =============================
platform win32 -- Python 3.9.13, pytest-8.1.1, pluggy-1.4.0
rootdir: c:\Users\42Maru\til\til\pytest\4_how_to_use_fixtures
collected 1 item

test_append.py .

============================== 1 passed in 0.02s ==============================


### 여러 fixtures 사용
여러 fixtures를 테스트 함수에 넣을 수도 있다.

In [2]:
!pytest test_append_multiple.py

============================= test session starts =============================
platform win32 -- Python 3.9.13, pytest-8.1.1, pluggy-1.4.0
rootdir: c:\Users\42Maru\til\til\pytest\4_how_to_use_fixtures
collected 1 item

test_append_multiple.py .                                                [100%]

============================== 1 passed in 0.01s ==============================


### fixtures 여러 번 호출
append_first()의 결과로 order에 'a'가 추가된다. 그리고 order를 재호출했을 때 그 값이 그대로 사용된다.

In [5]:
!pytest test_append_more_than_once.py

============================= test session starts =============================
platform win32 -- Python 3.9.13, pytest-8.1.1, pluggy-1.4.0
rootdir: c:\Users\42Maru\til\til\pytest\4_how_to_use_fixtures
collected 1 item

test_append_more_than_once.py .                                          [100%]

============================== 1 passed in 0.01s ==============================


### autouse fixtures
테스트에서 항상 append_first()가 자동으로 실행된다.

In [6]:
!pytest test_append_autouse.py

============================= test session starts =============================
platform win32 -- Python 3.9.13, pytest-8.1.1, pluggy-1.4.0
rootdir: c:\Users\42Maru\til\til\pytest\4_how_to_use_fixtures
collected 2 items

test_append_autouse.py ..                                                [100%]

============================== 2 passed in 0.01s ==============================


### scope

In [9]:
!pytest test_scope.py

============================= test session starts =============================
platform win32 -- Python 3.9.13, pytest-8.1.1, pluggy-1.4.0
rootdir: c:\Users\42Maru\til\til\pytest\4_how_to_use_fixtures
collected 2 items

test_scope.py FF                                                         [100%]

================================== FAILURES ===================================
__________________________________ test_ehlo __________________________________

smtp_connection = <smtplib.SMTP object at 0x000001FA9625B9A0>

    def test_ehlo(smtp_connection):
        response, msg = smtp_connection.ehlo()
        assert response == 250
        assert b"smtp.gmail.com" in msg
>       assert 0  # for demo purposes
E       assert 0

test_scope.py:5: AssertionError
__________________________________ test_noop __________________________________

smtp_connection = <smtplib.SMTP object at 0x000001FA9625B9A0>

    def test_noop(smtp_connection):
        response, msg = smtp_connection.noop()
     

아래 코드와 같이 scope를 dynamic하게 지정할 수 있다.

In [ ]:
def determine_scope(fixture_name, config):
    if config.getoption("--keep-containers", None):
        return "session"
    return "function"


@pytest.fixture(scope=determine_scope)
def docker_container():
    yield spawn_container()

### yield

setup이 늦게된 receiving_user부터 teardown이 수행된다.

In [9]:
!pytest -s test_emaillib.py

============================= test session starts =============================
platform win32 -- Python 3.9.13, pytest-8.1.1, pluggy-1.4.0
rootdir: c:\Users\42Maru\til\til\pytest\4_how_to_use_fixtures
collected 1 item

test_emaillib.py .receiving_user teardown!
sending_user teardown!


============================== 1 passed in 0.00s ==============================
